In [1]:
import geopandas as gpd
import pandas as pd
import os
import io
import requests
import boto3

This script intitate the dataframes that will store the different indicators calauclated for c4f cities.
- Input: georef file containing the list of cities' names and that we use for retrieving the boundaries files from the aws s3 (you can find more information about the georeferential-file in [this wiki page](https://github.com/wri/cities-urbanshift/wiki/Tutorial#administrative-boundaries))
- Output: A data frame containing the list of cities and sub-cities with their uniques identifies. This dataframe will be used as a based for integrating the indicators. It is stored in aws s3 bucket.

Run the script only when you want to initialize all the indicators.

# Load input data

In [2]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Fill initial indicator table with the list of features

In [3]:
# create empty df
cities_indicators = pd.DataFrame()

In [5]:
# populate the df with the list of geo identifiers
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    # get boundary area of interest
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_data = gpd.read_file(boundary_path)
    boundary_data = boundary_data[["geo_id","geo_level", "geo_name","geo_parent_name"]]
    
    # append to dataframe
    cities_indicators = pd.concat([cities_indicators, boundary_data])
    
    # get boundary unit of analysis
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_data = gpd.read_file(boundary_path)
    boundary_data = boundary_data[["geo_id","geo_level", "geo_name","geo_parent_name"]]
    
    # append to dataframe
    cities_indicators = pd.concat([cities_indicators, boundary_data])
    

0

 geo_name: BRA-Salvador
1

 geo_name: COD-Bukavu
2

 geo_name: COD-Uvira
3

 geo_name: COG-Brazzaville
4

 geo_name: COL-Barranquilla
5

 geo_name: ETH-Addis_Ababa
6

 geo_name: ETH-Dire_Dawa
7

 geo_name: KEN-Nairobi
8

 geo_name: MDG-Antananarivo
9

 geo_name: MEX-Mexico_City
10

 geo_name: MEX-Monterrey
11

 geo_name: RWA-Musanze


In [6]:
cities_indicators

,geo_id,geo_level,geo_name,geo_parent_name
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador
...,...,...,...,...
135,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze
136,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze
137,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze
138,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze


# Upload in aws s3

In [7]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [12]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [13]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')